<a href="https://colab.research.google.com/github/jishwithareddy/glowmodel_classification/blob/main/Copy_of_GLOW_Model_Final2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install normflows

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for normflows: filename=normflows-1.7.2-py2.py3-none-any.whl size=86917 sha256=29a462772fdd2990490e54aac932b1e0f6d39336247807e873b6c6b4b69ec12d
  Stored in directory: /root/.cache/pip/wheels/8a/a4/89/3e09f53a561355c45eccfebeffc07a0e34d36a3f41e3ef68a3
Successfully built normflows


In [ ]:
import torch
import torchvision as tv
import numpy as np
import normflows as nf
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from matplotlib import pyplot as plt
from tqdm import tqdm
import torch.nn as nn
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader

In [ ]:
# Set up model

# Define flows
L = 2
K = 16
torch.manual_seed(0)

input_shape = (1, 28, 28)  # MNIST images are grayscale
# channels = 1  # Only one channel for grayscale images

n_dims = np.prod(input_shape)
channels = 1
hidden_channels = 256
split_mode = 'channel'
scale = True
num_classes = 10  # MNIST has 10 classes (digits 0-9)
# Set up flows, distributions and merge operations
q0 = []
merges = []
flows = []
for i in range(L):
    flows_ = []

    for j in range(K):
       flows_ += [nf.flows.GlowBlock(channels * 2 ** (L + 1 - i), hidden_channels,
                             split_mode=split_mode, scale=scale)]

    flows_ += [nf.flows.Squeeze()]
    flows += [flows_]
    if i > 0:
        merges += [nf.flows.Merge()]
        latent_shape = (input_shape[0] * 2 ** (L - i), input_shape[1] // 2 ** (L - i),
                     input_shape[2] // 2 ** (L - i))
    else:
        latent_shape = (input_shape[0] * 2 ** (L + 1 - i), input_shape[1] // 2 ** (L - i),
                     input_shape[2] // 2 ** (L - i))
    q0 += [nf.distributions.ClassCondDiagGaussian(latent_shape, num_classes)]


# Construct flow model with the multiscale architecture
model = nf.MultiscaleFlow(q0, flows, merges)

/usr/local/lib/python3.10/dist-packages/torch/_tensor.py:738: UserWarning: torch.lu is deprecated in favor of torch.linalg.lu_factor / torch.linalg.lu_factor_ex and will be removed in a future PyTorch release.
LU, pivots = torch.lu(A, compute_pivots)
should be replaced with
LU, pivots = torch.linalg.lu_factor(A, compute_pivots)
and
LU, pivots, info = torch.lu(A, compute_pivots, get_infos=True)
should be replaced with
LU, pivots, info = torch.linalg.lu_factor_ex(A, compute_pivots) (Triggered internally at ../aten/src/ATen/native/BatchLinearAlgebra.cpp:1992.)
  LU, pivots, infos = torch._lu_with_info(


In [ ]:
enable_cuda = True
device = torch.device('cuda' if torch.cuda.is_available() and enable_cuda else 'cpu')

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Set batch size and other parameters
batch_size = 128


# Define data transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    # Add any other transformations as needed
])

# Load the MNIST dataset
mnist_dataset = datasets.MNIST('datasets/', train=True, download=True, transform=transform)

100%|██████████| 9912422/9912422 [00:00<00:00, 139875354.92it/s]

Extracting datasets/MNIST/raw/train-images-idx3-ubyte.gz to datasets/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 46358857.18it/s]


Extracting datasets/MNIST/raw/train-labels-idx1-ubyte.gz to datasets/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 82350667.37it/s]

Extracting datasets/MNIST/raw/t10k-images-idx3-ubyte.gz to datasets/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 6633192.47it/s]


Extracting datasets/MNIST/raw/t10k-labels-idx1-ubyte.gz to datasets/MNIST/raw



In [ ]:
# Define a custom function to filter the dataset
def filter_dataset(dataset, target_digit):
    filtered_data = [(x, y) for x, y in dataset if y == target_digit]
    return filtered_data

# We can reuse the filter_dataset function to filter datasets for 0 and 1
filtered_dataset_zeros = filter_dataset(mnist_dataset, 0)
filtered_dataset_ones = filter_dataset(mnist_dataset, 1)

# Create separate loaders for 0 and 1
train_loader_zeros = DataLoader(filtered_dataset_zeros, batch_size=batch_size, shuffle=True, drop_last=True)
train_loader_ones = DataLoader(filtered_dataset_ones, batch_size=batch_size, shuffle=True, drop_last=True)

# Create separate GLOW models for 0 and 1
model_zeros = nf.MultiscaleFlow(q0, flows, merges).to(device)
model_ones = nf.MultiscaleFlow(q0, flows, merges).to(device)

optimizer_zeros = torch.optim.Adamax(model_zeros.parameters(), lr=1e-3, weight_decay=1e-5)
optimizer_ones = torch.optim.Adamax(model_ones.parameters(), lr=1e-3, weight_decay=1e-5)

In [ ]:
max_iter=100

In [ ]:
def train_glow(model, loader, optimizer, max_iter):
    train_iter = iter(loader)
    loss_hist = []
    for i in tqdm(range(max_iter)):
        try:
            x, y = next(train_iter)
        except StopIteration:
            train_iter = iter(loader)
            x, y = next(train_iter)
        optimizer.zero_grad()
        loss = model.forward_kld(x.to(device), y.to(device))

        # 3. Notify if loss is NaN or Inf
        if torch.isnan(loss) or torch.isinf(loss):
            print(f"Loss has problematic value {loss.item()} at iteration {i}")
            break  # halt training

        loss.backward()
        optimizer.step()
        loss_hist.append(loss.detach().cpu().item())  # Ensure the loss is scalar
    return loss_hist

loss_hist_zeros = train_glow(model_zeros, train_loader_zeros, optimizer_zeros, max_iter)
loss_hist_ones = train_glow(model_ones, train_loader_ones, optimizer_ones, max_iter)

In [ ]:
print(dir(model_zeros))


In [91]:
def get_latent_mean(model, loader, num_classes=None):
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        latents = []
        for x, labels in loader:
            x = x.to(device)
            # If the model is class-conditional, we need to provide class labels `y`
            if num_classes is not None:
                # Create a one-hot encoding of the labels if necessary
                y = torch.zeros(labels.size(0), num_classes, device=device)
                y.scatter_(1, labels.unsqueeze(1).to(device), 1)
            else:
                y = None  # If not class-conditional, y should be None

            z = model(x, y) if num_classes is not None else model(x)
            latents.append(z.cpu())  # Move latent vectors to CPU memory
        latents = torch.cat(latents, dim=0)
        mean_latent = latents.mean(dim=0)

    # Print the mean latent vector
    print("Mean latent vector:", mean_latent)
    return mean_latent


In [92]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])

# Download and load the MNIST test dataset
mnist_test = MNIST(root='./data', train=False, download=True, transform=transform)
test_dataset_zeros = filter_dataset(mnist_test, 0)
test_dataset_ones = filter_dataset(mnist_test, 1)
test_loader_zeros = DataLoader(test_dataset_zeros, batch_size=batch_size, shuffle=True)
test_loader_ones = DataLoader(test_dataset_ones, batch_size=batch_size, shuffle=True)
print(dir(model_zeros))
num_classes = 10  # For MNIST, we have 10 classes
# mean_latent_zeros = get_latent_mean(model_zeros, train_loader_zeros, num_classes=num_classes)
# mean_latent_ones = get_latent_mean(model_ones, train_loader_ones, num_classes=num_classes)
mean_latent_zeros = get_latent_mean(model_zeros, train_loader_zeros, num_classes=num_classes)
print("Mean latent vector for zeros:", mean_latent_zeros)

mean_latent_ones = get_latent_mean(model_ones, train_loader_ones, num_classes=num_classes)
print("Mean latent vector for ones:", mean_latent_ones)


from torchvision.datasets import MNIST
from torchvision import transforms
from torch.utils.data import DataLoader
import torch

# Your `get_latent_mean` and `filter_dataset` functions should be defined above this snippet
# Your `model_zeros` and `model_ones` should be already defined and loaded

transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])

def classify(model, test_loader, mean_latent_target, other_mean_latent, num_classes=None):
    correct = 0
    total = 0
    with torch.no_grad():
        for x, labels in test_loader:
            x = x.to(device)
            labels = labels.to(device)  # Make sure labels are also moved to the device
            # If the model is class-conditional, we need to provide class labels `y`
            if num_classes is not None:
                # Ensure labels are LongTensor for scatter_
                labels = labels.long()
                # Create a one-hot encoding of the labels if necessary
                y = torch.zeros(labels.size(0), num_classes, device=device)
                y.scatter_(1, labels.unsqueeze(1), 1)
            else:
                y = None  # If not class-conditional, y should be None

            z = model(x, y) if num_classes is not None else model(x)

            # Ensure z has two dimensions, if it's currently 1D it's reshaped to 2D with the second dim being 1
            if z.dim() == 1:
                z = z.view(-1, 1)

            # Move mean latent vectors to device and add a batch dimension so they can be broadcasted
            mean_latent_target = mean_latent_target.to(device).unsqueeze(0)  # Broadcast shape: [1, feature_size]
            other_mean_latent = other_mean_latent.to(device).unsqueeze(0)    # Broadcast shape: [1, feature_size]

            # Now `z`, `mean_latent_target`, and `other_mean_latent` can be broadcast against each other
            dist_to_target = ((z - mean_latent_target)**2).sum(dim=1).sqrt()
            dist_to_other = ((z - other_mean_latent)**2).sum(dim=1).sqrt()

            # Predict target if closer to target mean latent
            predictions = (dist_to_target < dist_to_other).long()
            correct += predictions.sum().item()
            total += predictions.size(0)

    return correct / total


# Now we call the `classify` function with the `num_classes` argument
accuracy_zeros = classify(model_zeros, test_loader_zeros, mean_latent_zeros, mean_latent_ones, num_classes=num_classes)
accuracy_ones = classify(model_ones, test_loader_ones, mean_latent_ones, mean_latent_zeros, num_classes=num_classes)

print(f'Classification accuracy for 0: {accuracy_zeros * 100:.2f}%')
print(f'Classification accuracy for 1: {accuracy_ones * 100:.2f}%')


['T_destination', '__annotations__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_backward_hooks', '_backward_pre_hooks', '_buffers', '_call_impl', '_compiled_call_impl', '_forward_hooks', '_forward_hooks_always_called', '_forward_hooks_with_kwargs', '_forward_pre_hooks', '_forward_pre_hooks_with_kwargs', '_get_backward_hooks', '_get_backward_pre_hooks', '_get_name', '_is_full_backward_hook', '_load_from_state_dict', '_load_state_dict_post_hooks', '_load_state_dict_pre_hooks', '_maybe_warn_non_full_backward_hook', '_modules', '_named_members', '_non_persistent_buffers_set', '_parameters', '_register_load_state_dict_pre_hoo

In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(loss_hist_zeros, label="Zeros Loss")
plt.plot(loss_hist_ones, label="Ones Loss")
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Training Loss History')
plt.legend()
plt.show()